In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from kaggle_environments import evaluate, make, utils
import numpy as np # linear algebra

# Agent sample code

In [ ]:
%%writefile tiny_agent.py
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

def agent(obs_dict, config_dict):
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)

    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    
    #px, pc = row_col(player_head, configuration.columns)
    return "EAST"   

In [ ]:
%%writefile test_agent.py
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
##agent 


prev_step = None

def agent(obs_dict, config_dict):
    #print(config_dict)
    global prev_step
    
    legend = {
    1: "SOUTH",
    2: "NORTH",
    3: "EAST",
    4: "WEST"
    }
    
    
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    #print(observation)
    #print(configuration)

    player_index = observation.index
    player_goose = observation.geese[player_index]
    #print(player_index)
    #print(player_goose)

    player_head = player_goose[0]
    
    px, py = row_col(player_head, configuration.columns)
    
    
    foods = []
    for food in observation.food:
        x, y = row_col(food, configuration.columns)
        foods.append((x,y))

    #print(foods[0])
    fx, fy = foods[0]
    

    ##note that (0,0):= top left corner
    ##note x is vertical, y is horizontal
    
    action = 3
    if (fx > px) and (prev_step != 2):
        action = 1 #South
    if (fx < px) and (prev_step != 1):
        action = 2 #North
    if (fy > py) and (prev_step != 4):
        action = 3 #East
    if (fy < py) and (prev_step != 3):
        action = 4 #West
    
    
    prev_step = action
    return legend[action]##Action.East.name
        

# The Facts / Plan

### Facts
First things first let's get some facts out of the way. The game is set on a (7,11) grid, where if we go left on the left edge we reappear on the right edge aka. Donut World https://infinityplusonemath.wordpress.com/2017/02/18/asteroids-on-a-donut/. This means that whenever we take direction we need to acount for the symmetry of the world.

Our player/goose can only move in 4 direction (Up, Down, Left, Right) = (NORTH, SOUTH, WEST, EAST). This means that if we want to find things closest to our goose we should use the 1-norm/Manhattan distance https://en.wikipedia.org/wiki/Taxicab_geometry. To effectively find distances.

### Plan
The easiest and simplest "smart" goose to make is the "greedy" agent/goose. This just goes toward the closest reward/food without worrying about opponents or obstacles (not like we have any obstacles here).

We can easily make this agent with good reusable function for distance and direction calculation for more complicated agents later.

In [ ]:
%%writefile hungry_agent.py
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np

##Note that due to examples the x,y directions are flipped relative to normal games/apps
##we could just flip it if needed.
def donut_world(x,y):
    ##7/2 = 3.5 -> 4
    ##11/2 = 5.5 -> 6
    if x > 4 : x -= 4
    if x < -4 : x += 4
    if y > 6 : y -= 6
    if y < -6 : y += 6
    return (x,y)

def distance(v1, v2): ##v := (x,y)
    dx = (v1[0]-v2[0])
    dy = (v1[1]-v2[1])
    dx,dy = donut_world(dx,dy)
    return np.abs(dx) + np.abs(dy)

def find_closest(vectors, point):
    max_d = np.Inf
    close_point = vectors[0]   
    for v in vectors:
        d = distance(v, point)
        if d < max_d:
            max_d = d
            close_point = v
    return close_point, max_d
            

def from_to(v1, v2): 
    return donut_world(v2[0]-v1[0], v2[1]-v1[1])

def vector_add(v1,v2):
    return donut_world(v1[0]+v2[0], v1[1]+v2[1])

def vector_to_dir(vector):
    sgn0 = np.sign(vector[0])
    sgn1 = np.sign(vector[1])
    #vector = (sgn0, sgn1)
    if (sgn0 != 0) and (sgn1 != 0):
        #we need to choose which direction we want to go in
        return (sgn0, 0) #for simplicity choose vertical direction
    
    return (sgn0, sgn1)

legend2 = {
    (1,0): "SOUTH",
    (-1,0): "NORTH",
    (0,1): "EAST",
    (0,-1): "WEST"
}

prev_step = None

def agent(obs_dict, config_dict):
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player = (px, py) = row_col(player_head, configuration.columns)
    print("step = ", observation.step)
    global legend2
    global prev_step
    
    foods =[]
    for food in observation.food:
        foods.append(row_col(food, configuration.columns))
    
    ##we find the closest food
    close_food = (0,0)
    close_food, dist = find_closest(foods, player)
    
    print(f"closest food at [{close_food[0]}, {close_food[1]}]; at distance [{dist}]")
    print(player)
    print(close_food)
    delta = from_to(player, close_food)
    print(f"delta = {delta}")
    
    step = vector_to_dir(delta)
    print("step towards ", step)
    if (prev_step != None) and vector_add(prev_step, step) == (0,0):
        ##we can't move inwards to our body
        step = (step[1], step[0])
    
    prev_step = step
    return legend2[step]



# Next Steps (Still deterministic)

We would preferably want to make an agent/goose that knows about the world. So we need to make it have a "memory" of the grid it lives in. We can store a Matrix of encoded numbers to know the current state of the board/world.

Next we could consider all goose bodies as obstacles. This means we could apply very standard pathfinding algorithms (A*) to find our way to our objective even with many goose being in the way.

Next we could try to determine what other gooses/players can/would do so we can account for their actions. eg.: we their goose is closer to our target why bother going for it. Or to block their way of reaching their goal.

But preferably we would want to train an agent with the above defined world state for it to find out what is the best action to do.

In [ ]:
%%writefile better_agent.py
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np

##We will mark food on the world by a 1
##player position = 5
##enemy goose by a -1
##enemy potential points by a -2


##Note that due to examples the x,y directions are flipped relative to normal games/apps
##we could just flip it if needed.
def donut_world(x,y):
    ##7/2 = 3.5 -> 4
    ##11/2 = 5.5 -> 6
    if x > 4 : x -= 4
    if x < -4 : x += 4
    if y > 6 : y -= 6
    if y < -6 : y += 6
    return (x,y)

def distance(v1, v2): ##v := (x,y)
    dx = (v1[0]-v2[0])
    dy = (v1[1]-v2[1])
    dx,dy = donut_world(dx,dy)
    return np.abs(dx) + np.abs(dy)

def find_closest(vectors, point):
    max_d = np.Inf
    close_point = vectors[0]   
    for v in vectors:
        d = distance(v, point)
        if d < max_d:
            max_d = d
            close_point = v
    return close_point, max_d
            

def from_to(v1, v2): 
    return donut_world(v2[0]-v1[0], v2[1]-v1[1])

def vector_add(v1,v2):
    return donut_world(v1[0]+v2[0], v1[1]+v2[1])

legend2 = {
    (1,0): "SOUTH",
    (-1,0): "NORTH",
    (0,1): "EAST",
    (0,-1): "WEST"
}
 


def vector_to_dir(vector):
    sgn0 = np.sign(vector[0])
    sgn1 = np.sign(vector[1])
    #vector = (sgn0, sgn1)
    if (sgn0 != 0) and (sgn1 != 0):
        #we need to choose which direction we want to go in
        return (sgn0, 0) #for simplicity choose vertical direction
    
    return (sgn0, sgn1)
        

prev_step = None
def agent(obs_dict, config_dict):
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player = (px, py) = row_col(player_head, configuration.columns)
    print("step = ", observation.step)
    global legend2
    global prev_step
   
    world = np.zeros((configuration.rows, configuration.columns))
    for p in player_goose:
        pos = row_col(p, configuration.columns)
        world[pos[0], pos[1]] = 5
    
    foods =[]
    for food in observation.food:
        foods.append(row_col(food, configuration.columns))
        world[foods[-1][0], foods[-1][1]] = 1
    
    for i, goose in enumerate(observation.geese): ##looping through all geese
        if i == player_index: continue ##that's use -> pointless (for now)
        if len(goose) == 0: continue ##enemy doesn't exist
            
        for j in goose:
            x,y = row_col(j, configuration.columns)
            world[x,y] = -1
            
    #print(world)
    
    ##we find the closest food
    close_food = (0,0)
    close_food, dist = find_closest(foods, player)
    
    print(f"closest food at [{close_food[0]}, {close_food[1]}]; at distance [{dist}]")
    delta = from_to(player, close_food)
    print(f"delta = {delta}")
    
    step = vector_to_dir(delta)
    print("step towards ", step)
    if (prev_step != None) and vector_add(prev_step, step) == (0,0):
        ##we can't move inwards to our body
        step = (step[1], step[0])
        
    prev_step = step
    return legend2[step]
    

# Running the Environment / FIGHT!!!

In [ ]:
##testing environment
env = make("hungry_geese", debug=True)

In [ ]:
env.reset()
env.run(["better_agent.py", "hungry_agent.py"])
env.render(mode="ipython", width=500, height=450)